In [1]:
import pandas as pd
from collections import Counter
from copy import copy
# Lists of given courses and TAs
TAs = ['Aya','Alaa','Mohammed','Bassam','Ali','Ahmed','Arwa']
courses = ['205','207','217','479']
# Preferred courses for each TA dataframe (1 being highest and 4 lowest)
pref = pd.DataFrame({'205': [2,3,2,4,2,3,1], '207': [3,2,1,3,3,1,4], '217':[1,1,4,2,2,4,3], '479':[4,4,3,1,1,2,2]})
pref.index = TAs
pref

,205,207,217,479
Aya,2,3,1,4
Alaa,3,2,1,4
Mohammed,2,1,4,3
Bassam,4,3,2,1
Ali,2,3,2,1
Ahmed,3,1,4,2
Arwa,1,4,3,2


In [2]:
# Skilled courses for each TA dataframe (1 being highest and 4 lowest)
skill = pd.DataFrame({'205': [1,1,3,1,2,4,3], '207': [3,4,2,4,1,3,1], '217':[2,2,1,2,4,1,4], '479':[4,3,4,3,3,2,2]})
skill.index = TAs
skill

,205,207,217,479
Aya,1,3,2,4
Alaa,1,4,2,3
Mohammed,3,2,1,4
Bassam,1,4,2,3
Ali,2,1,4,3
Ahmed,4,3,1,2
Arwa,3,1,4,2


In [3]:
# dict to control which course each TA can assign to
courses_available = {ta:courses for ta in TAs}
# waiting list of TAs that were assigned on each iteration
waiting_list = []
# dict to store created assigning
assigned = {}
# variable to count number of iterations
count = 0

In [4]:
# while not all TAs have been assigned
while len(waiting_list)<len(TAs):

    # TA is being assigned
    for ta in TAs:
        # each ta is assigned to it's preferred course from it's list
        if ta not in waiting_list:
            #List of courses
            co = courses_available[ta]
            # Return course with most preferred for a specific TA
            best_choice = pref.loc[ta][pref.loc[ta].index.isin(co)].idxmin()
            # Add to the dict the key(TA, course assigned) with value (rank in preferred, rank in skilled)
            assigned[(ta, best_choice)]=(pref.loc[ta][best_choice],
                                                 skill.loc[ta][best_choice])

    """
        If one course has more than one assigning 
        it will choose the skilled option
    """
    # return count of each assigned course
    countt = Counter([key[1] for key in assigned.keys()])

    # cycle to choose the best options
    for c in countt.keys():
        # setting max. no. of course assigning here
        if countt[c] in range(1,3):
            #return list of tuples after being sorted with their skilled rank and then remove the first tuple [0]
            # as being best choice to assign
            to_drop = sorted({pair: assigned[pair] for pair in assigned.keys() 
                    if c in pair}.items(), 
                   key=lambda x: x[1][1])[1:]

            for p_to_drop in to_drop:
                #delete the to_drop repeated assigning from assigned dict if it has least skilled compared to the preferred
                if (p_to_drop[1][1])>2:
                    del assigned[p_to_drop[0]]
                
                # copy the available courses for this TA
                    cour = copy(courses_available[p_to_drop[0][0]])
                # remove the repeated course from the TA's available courses
                    cour.remove(p_to_drop[0][1])
                # after removing the declined course, switch the copied available course and the original
                    courses_available[p_to_drop[0][0]] = cour
                    continue
            
                
    # courses that has successfully been assigned added to the waiting list 
    waiting_list = [t[0] for t in assigned.keys()]
    
    # update counter
    count+=1

In [5]:
assigned

{('Aya', '217'): (1, 2),
 ('Alaa', '217'): (1, 2),
 ('Mohammed', '207'): (1, 2),
 ('Ali', '205'): (2, 2),
 ('Ahmed', '479'): (2, 2),
 ('Bassam', '217'): (2, 2),
 ('Arwa', '479'): (2, 2)}